In [1]:
# import dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import os
import pandas as pd
import time
import re

In [2]:
#pointing to the directory where chromedriver exists
executable_path = {"executable_path":"chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless = False)

In [3]:
# visiting the page
url = "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(12)

In [4]:
# using bs to write it into html
html = browser.html
soup = bs(html,"html.parser")

In [5]:
# scrape headlines
class_title = soup.find(class_= 'list_text')
news_title = class_title.find('a').text.strip()
news_para = soup.find("div", class_="article_teaser_body").text
print(f"Title: {news_title}")
print(f"Para: {news_para}")

Title: NASA Invites Public to Share Excitement of Mars 2020 Perseverance Rover Launch
Para: There are lots of ways to participate in the historic event, which is targeted for July 30.


In [6]:
# visit the JPL website
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)


In [7]:
# using bs to write it into html
html_img = browser.html
soup_img = bs(html_img, "html.parser")

In [8]:
# scrape the featured image
image_url = soup_img.find('div',class_='carousel_container').article.footer.a['data-fancybox-href']
featured_image_url = f'https://www.jpl.nasa.gov{image_url}'
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17793_ip.jpg


In [9]:
# visit the Mars weather twitter page
weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)

In [10]:
#get mars weather's latest tweet from the website
weather_html = browser.html
weather_soup = bs(weather_html, 'html.parser')

mars_weather = weather_soup.find('span', text=re.compile("InSight")).text.strip()
print(mars_weather)

InSight sol 588 (2020-07-22) low -89.2ºC (-128.6ºF) high -11.0ºC (12.2ºF)
winds from the W at 5.7 m/s (12.7 mph) gusting to 18.9 m/s (42.3 mph)
pressure at 7.90 hPa


In [11]:
# visit space facts and scrap the mars facts table
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)
mars_data = pd.read_html(facts_url)
mars_data = pd.DataFrame(mars_data[0])
mars_facts = mars_data.to_html(header = False, index = False)
print(mars_facts)

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [12]:
# scrape images of Mars' hemispheres from the USGS site
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hemi_dicts = []

for i in range(1,9,2):
    hemi_dict = {}
    
    browser.visit(mars_hemisphere_url)
    time.sleep(1)
    hemispheres_html = browser.html
    hemispheres_soup = bs(hemispheres_html, 'html.parser')
    hemi_name_links = hemispheres_soup.find_all('a', class_='product-item')
    hemi_name = hemi_name_links[i].text.strip('Enhanced')
    
    detail_links = browser.find_by_css('a.product-item')
    detail_links[i].click()
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    time.sleep(1)
    browser.windows.current = browser.windows[-1]
    hemi_img_html = browser.html
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()
    
    hemi_img_soup = bs(hemi_img_html, 'html.parser')
    hemi_img_path = hemi_img_soup.find('img')['src']

    print(hemi_name)
    hemi_dict['title'] = hemi_name.strip()
    
    print(hemi_img_path)
    hemi_dict['img_url'] = hemi_img_path

    hemi_dicts.append(hemi_dict)


C:\Users\jerod\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:533: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  warnings.warn(


Cerberus Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


In [13]:
print(hemi_dicts)

[{'title': 'Cerberus Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [14]:
browser.quit()